In [25]:
import numpy as np
import pandas as pd

In [26]:
df = pd.read_csv('./ml-25m/genome-scores.csv')

In [27]:
print(df.head())

   movieId  tagId  relevance
0        1      1    0.02875
1        1      2    0.02375
2        1      3    0.06250
3        1      4    0.07575
4        1      5    0.14075


In [28]:
sorted_relevance = df[df['relevance'] > 0.5].sort_values(by=['movieId', 'relevance'], ascending=[True, False])

In [29]:
movie_id_names = pd.read_csv('./ml-25m/movies.csv')

In [30]:
print(movie_id_names.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [31]:
movie_tags = pd.merge(sorted_relevance, movie_id_names, on='movieId')

In [32]:
tags = pd.read_csv('./ml-25m/genome-tags.csv')

In [33]:
movie_tags = pd.merge(movie_tags, tags, on='tagId')

In [34]:
print(movie_tags.head())

   movieId  tagId  relevance             title  \
0        1   1036    0.99925  Toy Story (1995)   
1        1    244    0.99875  Toy Story (1995)   
2        1    786    0.99575  Toy Story (1995)   
3        1    589    0.98575  Toy Story (1995)   
4        1     64    0.98425  Toy Story (1995)   

                                        genres                 tag  
0  Adventure|Animation|Children|Comedy|Fantasy                toys  
1  Adventure|Animation|Children|Comedy|Fantasy  computer animation  
2  Adventure|Animation|Children|Comedy|Fantasy     pixar animation  
3  Adventure|Animation|Children|Comedy|Fantasy     kids and family  
4  Adventure|Animation|Children|Comedy|Fantasy           animation  


In [35]:
simple_movie_tags = movie_tags.groupby('movieId').agg({
    'tag': lambda x: '|'.join(x), 
    'title': 'first',
    'genres': 'first'
}).reset_index()

In [36]:
links = pd.read_csv('./ml-25m/links.csv')

In [37]:
simple_movie_tags_with_ids = pd.merge(simple_movie_tags, links, on='movieId')

In [38]:
simple_movie_tags_with_ids = simple_movie_tags_with_ids[['imdbId', 'title', 'genres', 'tag']]

In [39]:
simple_movie_tags_with_ids.head(5)

,imdbId,title,genres,tag
0,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toys|computer animation|pixar animation|kids a...
1,113497,Jumanji (1995),Adventure|Children|Fantasy,adventure|children|fantasy|kids|special effect...
2,113228,Grumpier Old Men (1995),Comedy|Romance,sequel|good sequel|sequels|comedy|original|gun...
3,114885,Waiting to Exhale (1995),Comedy|Drama|Romance,women|chick flick|divorce|girlie movie|romanti...
4,113041,Father of the Bride Part II (1995),Comedy,sequel|good sequel|father daughter relationshi...


In [40]:
simple_movie_tags_with_ids['imdbId'] = simple_movie_tags_with_ids['imdbId'].apply(lambda x: 'tt' + str(x).zfill(7))

In [41]:
simple_movie_tags_with_ids.to_csv('./movie_tags.csv', index=False)

In [42]:
import sqlite3
conn = sqlite3.connect('../fastapi/query/movie.db')
cursor = conn.cursor()

In [43]:
movie_id_in_db = pd.read_sql_query('SELECT MovieID FROM Movie', conn)

In [20]:
# delete from simple_movie_tags_with_ids where imdbId not in (select imdbId from Movie);
simple_movie_tags_with_ids_in_database = simple_movie_tags_with_ids[simple_movie_tags_with_ids['imdbId'].isin(movie_id_in_db['MovieID'])]

In [21]:
simple_movie_tags_with_ids_in_database.to_csv('./movie_tags_in_database.csv', index=False)